## Moteur de recherche sur les données médicales 

Dans ce projet, nous allons construire un moteur de recherche appliqué aux données médicales issues du challenge TREC-COVID. Afin de respecter la logique pédagogique du cours de Recherche d'Information, nous suivrons une approche progressive :

Du modèle le plus simple vers les modèles avancés, en analysant à chaque étape les performances obtenues sur nos données réelles.

### Notre objectif 
**Mettre en œuvre, comparer et analyser différents modèles de recherche d'information, en appliquant l'état de l'art directement sur le corpus TREC-COVID Round 1, un ensemble de documents scientifiques portant sur le COVID-19.**


Nous allons commencer par charger les données (ici les documents sur le COVID), puis les requêtes et les qrels pour ces requêtes 

Nous utilisons la bibliothèque ir_datasets, qui fournit un accès structuré aux documents, aux requêtes (queries) et aux jugements de pertinence (qrels). Cela nous permet de :

- Charger directement le Round 1 du corpus, sans passer par un téléchargement manuel.

- Éviter le scraping des données sur les liens comme :

     - https://ir.nist.gov/covidSubmit/data/topics-rnd1.xml (pour les requêtes)

     - https://ir.nist.gov/covidSubmit/data/qrels-rnd1.txt (pour les qrels)



#### La structure finale de nos données 
- Documents : Articles scientifiques (titre, résumé, etc.)

- Requêtes : Questions médicales posées dans le cadre du challenge

- Qrels : Fichier de pertinence qui indique quels documents sont jugés pertinents pour chaque requête


### Chargement des données grâce à la librairie ir_dataset

In [1]:
import pandas as pd
import ir_datasets
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from tqdm import tqdm

# Charger uniquement round1
dataset = ir_datasets.load("cord19/trec-covid/round1")

# Ici nous avons juste un objet fourni par la librairie ir_datasets, 
# qui propose une interface pour accéder aux documents, requêtes, jugements de pertinence, etc.

Nous allons donc maintenant récupérer les documents de ce dataset que nous venons de charger afin de pouvoir les avoir sous forme de dataFrame ainsi on pourra voir le shape, le header...

In [2]:
# Préparation
docs = []
for doc in tqdm(dataset.docs_iter(), desc="Chargement des documents"):
    docs.append({
        "doc_id": doc.doc_id,
        "title": doc.title,
        "abstract": doc.abstract
    })

# Conversion en DataFrame
df_docs = pd.DataFrame(docs)

# Affichage
print(df_docs.shape)
print(df_docs.columns)
df_docs.head()


Chargement des documents: 51078it [00:00, 175144.85it/s]

(51078, 3)
Index(['doc_id', 'title', 'abstract'], dtype='object')


,doc_id,title,abstract
0,xqhn0vbp,Airborne rhinovirus detection and effect of ul...,"BACKGROUND: Rhinovirus, the most common cause ..."
1,gi6uaa83,Discovering human history from stomach bacteria,Recent analyses of human pathogens have reveal...
2,le0ogx1s,A new recruit for the army of the men of death,"The army of the men of death, in John Bunyan's..."
3,fy4w7xz8,Association of HLA class I with severe acute r...,BACKGROUND: The human leukocyte antigen (HLA) ...
4,0qaoam29,A double epidemic model for the SARS propagation,BACKGROUND: An epidemic of a Severe Acute Resp...


### Chargement des queries et des qrels 

Nous allons nous servir de l'objet **dataset** chargé pour obtenir les queries et les qrels 

In [3]:
## Chargement des queries 

queries_iter = dataset.queries_iter()  # ou dataset.queries


Nous pouvons maintenant transformer en dataFrame comme avec les documents précédement 

In [4]:
first_query = next(dataset.queries_iter())
print(first_query)


TrecQuery(query_id='1', title='coronavirus origin', description='what is the origin of COVID-19', narrative="seeking range of information about the SARS-CoV-2 virus's origin, including its evolution, animal source, and first transmission into humans")


In [5]:
queries = []
for q in dataset.queries_iter():
    queries.append({
        "query_id": q.query_id,
        "title": q.title,
        "description": q.description,
        "narrative": q.narrative
    })

queries_df = pd.DataFrame(queries)
queries_df.head()


,query_id,title,description,narrative
0,1,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
1,2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
2,3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
3,4,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
4,5,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...


In [6]:
## Chargement des qurels 

queries_iter = dataset.qrels_iter()

On procède comme précédement avec les queries 

In [7]:
qrels = []
for qrel in dataset.qrels_iter():
    qrels.append({
        "query_id": qrel.query_id,
        "doc_id": qrel.doc_id,
        "relevance": qrel.relevance
    })

qrels_df = pd.DataFrame(qrels)
qrels_df.head()


,query_id,doc_id,relevance
0,1,010vptx3,2
1,1,02f0opkr,1
2,1,04ftw7k9,0
3,1,05qglt1f,0
4,1,0604jed8,0


### Nettoyage simple

Nous allons commencer par effectuer un nettoyage simple pour le moment 

- Suppression des NAN
- Concatenation **title + abstract**
- Tokenisation simple 

### Préparation des outils 

In [8]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

tokenizer = RegexpTokenizer(r'\w+')  # Garde uniquement les mots alphanumériques
stop_words = ENGLISH_STOP_WORDS


### Fonction de nettoyage simple 

In [9]:
def clean_text(text):
    tokens = tokenizer.tokenize(text.lower())  # minuscules + tokenisation
    return [token for token in tokens if token not in stop_words] # On garde ceux qui ne sont pas dans la liste des stopwords


### Appliquons le nettoyage sur les documents (corpus)

In [ ]:

## On combine le titre et l'abstract 
documents_df = pd.DataFrame([
    {
        "doc_id": doc.doc_id,
        "text": (doc.title or "") + " " + (doc.abstract or "")
    }
    for doc in dataset.docs_iter()
])

# Appliquer le nettoyage
documents_df["tokens"] = documents_df["text"].fillna("").apply(clean_text)



In [14]:
documents_df.head(3)

,doc_id,text,tokens
0,xqhn0vbp,Airborne rhinovirus detection and effect of ul...,"[airborne, rhinovirus, detection, effect, ultr..."
1,gi6uaa83,Discovering human history from stomach bacteri...,"[discovering, human, history, stomach, bacteri..."
2,le0ogx1s,A new recruit for the army of the men of death...,"[new, recruit, army, men, death, army, men, de..."


#### Nettoyage des requêtes

In [13]:
queries_df["tokens"] = queries_df["title"].fillna("").apply(clean_text)


In [15]:
queries_df.head(3)

,query_id,title,description,narrative,tokens
0,1,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...,"[coronavirus, origin]"
1,2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...,"[coronavirus, response, weather, changes]"
2,3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...,"[coronavirus, immunity]"


### Modèle binaire 

Nous allons implémenter un modèle binaire ici et voir ce qui en est des performances, nous terminerons par des commentaire à propos de ce modèle 

- Index inversé (mot et liste des documents où il apparait)
- Requête en mode booléen 
- Recherche des documents concernant certains termes 

Nous ferons une évaluation simple (couverture de la requête)

### Modèle sac de mots 

- Matrice document-term avec présence (0/1)
- Similarité avec la requête via le nombre de mots communs 

Nous ferons une comparaison naive (cosine binaire)


### Fréquence des mots (Term Frequency)

- Poids selon le nombre d’occurrences dans le document.

- Matrice pondérée.

- Cosine similarity avec une requête vectorisée également.

### TF-IDF (Term Frequency-Inverse Document Frequency)

- Ici nous ajoutons l'aspect de rareté au moteur précédent (TF)
-  Nous allons faire une évaluation (Précision, rappel, nDCG, MRR (découvert lors des recherches, qui permet de savoir si le premier document est toujours celui qui est pertinent))

**MRR** : Mean Reciprocal Rank, si le document pertinent est à la position 1 alors le score est de 1, si c'est en deuxième, le score est de 1/2 ainsi de suite. Puis on fait la moyenne de tous ces scores ce qui donne le MRR global, si cela vaut 1 alors le moteur de recherche met toujours le document pertinent comme premier document 

### Après les évaluations, nous allons commencer à arranger notre moteur TF-IDF en faisant d'autres prétraitements des données, nous allons également faire des ajustements sur ce modèle TF-IDF avant de passer à un modèle avancé 